In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.ensemble import AdaBoostRegressor as Ada
from sklearn.feature_extraction.text import CountVectorizer as CV
from sklearn.neural_network import MLPRegressor as MP


Load in the train and test data....

In [ ]:
train = pd.read_csv("/kaggle/input/commonlitreadabilityprize/train.csv")
test = pd.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")


Brief EDA...we know that some is potentially irrelevant ie: copyright and error, as they're not included in the testing/scoring sets.

In [ ]:
train.head()
print("Train length ",len(train))
test.head()
print("Test length ", len(test))
target = train["target"]
#Create submission file
sub = pd.DataFrame(test["id"])

Setup a simple Ada Boost training

In [ ]:
A = Ada(random_state = 42)


Simple Feature Engineer

In [ ]:
train['exerpt_len'] = train.excerpt.str.len()

test['exerpt_len'] = test.excerpt.str.len()


In [ ]:
C=CV(strip_accents='unicode')

train_dict = C.fit_transform(train.excerpt.append(test.excerpt))
names = C.get_feature_names()
#Drop out all the possible conflicting column headings....
new_test = test.drop(["url_legal","license","id","excerpt"],axis=1)
new_train = train.drop(["url_legal","license","id","excerpt","target","standard_error"],axis=1)
#...join together
train_dict=pd.DataFrame(train_dict.toarray(),columns=names)
test_dict=pd.DataFrame(C.transform(test.excerpt).toarray(),columns=names)

name_len = pd.DataFrame(names)
name_len["len"] = name_len[0].str.len()
ndw=[]
awl=[]
greater_than_6 = []
most =[]

#Looking at word complexity etc...
for i in range(len(train)): #Subtly different length....
    check = train_dict.iloc[i]      #Get the line
    num_dist_word = check[check>0]  #Anything with a count >0
    ndw.append(len(num_dist_word))  #How long is that list
    len_ndw = num_dist_word.index.str.len()   #How long is each word in that list
    awl.append(sum(len_ndw) / len(len_ndw)) 
    junk = len_ndw>6    #test to see how many words are >6 ??
    greater_than_6.append(len(junk.nonzero()[0]))#Count TRUE
    stuff = len_ndw.value_counts()#Find the most popular word length
    most.append(stuff.index[0])
train['num_dist_word'] = ndw
train['avg_word_len'] = awl
train['g_t_6'] = greater_than_6
train['most_pop_len'] = most
#repetition...could have made a func???
ndw=[]
awl=[]
greater_than_6 = []
most =[]

for i in range(len(test_dict)):
    check = test_dict.iloc[i]
    num_dist_word = check[check>0]
    ndw.append(len(num_dist_word))
    len_ndw = num_dist_word.index.str.len()
    awl.append(sum(len_ndw) / len(len_ndw))
    junk = len_ndw>6
    greater_than_6.append(len(junk.nonzero()[0]))
    stuff = len_ndw.value_counts()
    most.append(stuff.index[0])
test['num_dist_word'] = ndw
test['avg_word_len'] = awl
test['g_t_6'] = greater_than_6
test['most_pop_len'] = most
#new_test = new_test.join(test_dict)
#new_train = new_train.join(train_dict)

Train and test....

In [ ]:
#Drop out all the possible conflicting column headings....
new_test = test.drop(["url_legal","license","id","excerpt"],axis=1)
new_train = train.drop(["url_legal","license","id","excerpt","standard_error"],axis=1)

#Treat target separate so we can convert back easier??
target = new_train.pop('target')
#Normalisation
# copy the data
new_train_scaled = new_train.copy()
  
# apply normalization techniques
for column in new_train_scaled.columns:
    new_train_scaled[column] = (new_train_scaled[column] - new_train_scaled[column].min()) / (new_train_scaled[column].max() - new_train_scaled[column].min())    

# copy the data
new_test_scaled = new_test.copy()
  
# apply normalization
for column in new_test_scaled.columns:
    #Use the original new_train values
    new_test_scaled[column] = (new_test_scaled[column] - new_train[column].min()) / (new_train[column].max() - new_train[column].min())    

t_min = target.min()
t_max = target.max()

target_scaled = (target - t_min) / (t_max - t_min)    


A = Ada(random_state = 42)

A.fit(new_train,target_scaled)

"""
#Create Keras model
len_in = new_train_scaled.shape[1]
len_in_2 = (len_in*2) *2

model = Sequential()
model.add(layers.Dense(len_in_2,activation="relu",input_dim=len_in))
model.add(layers.Dense(len_in_2,activation="relu"))
#model.add(layers.SimpleRNN(len_in_2))
#model.add(layers.Dense(len_in_2,activation="relu"))
model.add(layers.Dense(1,activation="sigmoid"))

model.compile(optimizer='Adam',loss='mse', metrics=[tf.keras.metrics.RootMeanSquaredError()])

model.fit(new_train_scaled,target_scaled,batch_size=1, epochs=10, shuffle=False)
"""
model = MP(activation='tanh',random_state = 42, hidden_layer_sizes=(100,100,25),verbose=1,solver='adam',)
model.fit(new_train_scaled, target_scaled)

A_pred = A.predict(new_test_scaled)

NN_pred = model.predict(new_test_scaled)
#Stack them??
NN_pred = (A_pred + NN_pred)/2

NN_pred_rescale = (NN_pred * (t_max - t_min)) + t_min
#**Need to rescale output to remove normalisation

#sub["target"] = A.predict(new_test)
sub["target"] = NN_pred_rescale


sub.to_csv("submission.csv", index=False)